In [2]:
import re
import json
import pandas as pd
from glob import glob
from os.path import join,basename
from msp_tables import prepare_tables, log
# make things prettier
from rich import print
from rich.progress import track, Progress

log.setLevel("ERROR")

bidsdir = '/mnt/bulk-vega/paulkuntke/mspaths'



First we filter out all available patient-IDs as well as the dates of examination

In [3]:
mpis = [ basename(sub).replace('sub-','')  for sub in glob(join(bidsdir, 'sub-*'))]

Read all the session-dates - as defined in the BIDS-Structure

In [4]:
sessionfiles = glob(join(bidsdir, 'sub-*', 'sub-*_sessions.tsv'))

sessions_df = pd.DataFrame()

subject_pattern = re.compile(r"sub-(\d+)_")


for sesfile in track(sessionfiles):
    df = pd.read_csv(sesfile, delimiter='\t')
    df["subject"] = re.findall(subject_pattern, basename(sesfile))[0]
    sessions_df = pd.concat((sessions_df, df))

del(df)
del(sessionfiles)

sessions_df.acq_time = pd.to_datetime(sessions_df.acq_time)



Output()

In [5]:
sessions_df.drop_duplicates()

,session_id,acq_time,subject
0,ses-001,2019-05-03,100052186
1,ses-002,2020-02-22,100052186
0,ses-001,2017-02-09,100006427
1,ses-002,2017-06-08,100006427
0,ses-001,2020-11-08,500002712
...,...,...,...
1,ses-002,2017-11-24,100003594
2,ses-003,2019-12-29,100003594
0,ses-001,2019-06-17,500003493
1,ses-002,2020-07-10,500003493


Now- lets read all tables ( _all_ is defined in `column_names.json`)

In [6]:
with open("column_names.json", "r") as file:
    table_data = json.load(file)
prepared_tables = prepare_tables('888MS001 & 888MS002/Data Tables/', bidsdir, table_data)
prepared_tables_hc = prepare_tables('888MS005/Data Tables/', bidsdir, table_data)

/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain Chars that are not readable with default utf-8 codepage
/home/paulkuntke/mspaths/msp_tables.py:74: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  f_df = pd.read_csv(f, encoding="cp1252") # Some tables contain 

In [7]:
for k, df in prepared_tables_hc.items():
    print(k)
    display(df)

MSPT Sociodemographics

,mpi,site,study_id,encdate,sex,race_org,ethnic,agesymp,agediag,file
0,100025931,275.0,5.0,1.517817e+09,female,black_or_african_american,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v013.csv
1,100026381,275.0,5.0,1.515330e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v013.csv
2,100027750,275.0,5.0,1.515852e+09,female,white,unknown,1.0,1.0,mspt_sociodemographics_888MS005_v013.csv
3,100027864,275.0,5.0,1.514634e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v013.csv
4,100028177,275.0,5.0,1.514797e+09,male,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v013.csv
...,...,...,...,...,...,...,...,...,...,...
1971,100051035,275.0,5.0,1.549020e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v008.csv
1972,100051191,275.0,5.0,1.549273e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v008.csv
1973,100051507,275.0,5.0,1.549799e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v008.csv
1974,100051532,235.0,5.0,1.549817e+09,female,white,not_hispanic_or_latino,1.0,1.0,mspt_sociodemographics_888MS005_v008.csv


MSPT Patientcharacteristics

,mpi,site,study_id,encdate,living,insuranc,employ,educ,pdds_cat,pdds_scr,mstype,wkaid,wkaid_hd,relapses,rel_stat,file
0,100025931,275.0,5.0,1.517817e+09,home_without_assistance,private,full_time,16.0,normal,0.0,Relapsing Remitting MS,NaN,NaN,0.0,NaN,mspt_patient characteristics_888MS005_v010.csv
1,100026381,275.0,5.0,1.515330e+09,home_without_assistance,other,full_time,16.0,normal,0.0,Relapsing Remitting MS,NaN,NaN,0.0,NaN,mspt_patient characteristics_888MS005_v010.csv
2,100027750,275.0,5.0,1.515852e+09,home_with_assistance,other,full_time,12.0,normal,0.0,Relapsing Remitting MS,NaN,NaN,0.0,NaN,mspt_patient characteristics_888MS005_v010.csv
3,100027750,275.0,5.0,1.547289e+09,home_without_assistance,private,full_time,NaN,normal,0.0,NaN,NaN,NaN,0.0,NaN,mspt_patient characteristics_888MS005_v010.csv
4,100027864,275.0,5.0,1.514634e+09,home_without_assistance,medicaid,unemployed,17.0,normal,0.0,Relapsing Remitting MS,NaN,NaN,0.0,NaN,mspt_patient characteristics_888MS005_v010.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,500012488,396.0,5.0,1.595764e+09,home_without_assistance,public_state,full_time,10.0,NaN,NaN,No CIS or MS Diagnosis,NaN,NaN,NaN,married,mspt_patient characteristics_888MS005_v013.csv
3481,500012765,326.0,5.0,1.603273e+09,home_without_assistance,free_social_healthcare,full_time,12.0,NaN,NaN,No CIS or MS Diagnosis,NaN,NaN,NaN,married,mspt_patient characteristics_888MS005_v013.csv
3482,500012785,326.0,5.0,1.603878e+09,home_without_assistance,free_social_healthcare,full_time,14.0,normal,0.0,Clinically Isolated Syndrome,NaN,NaN,0.0,married,mspt_patient characteristics_888MS005_v013.csv
3483,500012786,326.0,5.0,1.603879e+09,home_without_assistance,private_self,disabled,5.0,normal,0.0,Clinically Isolated Syndrome,NaN,NaN,0.0,divorced,mspt_patient characteristics_888MS005_v013.csv


EMR Sociodemographics

...

,mpi,site,study_id,vstype,nm_strt,nm_end,age,sh_dr_fr,sh_dr_ct,sh_dr_yr,sh_stre,sh_mod,sh_mild,sh_sp,sh_sp_yr,file
0,100025931,275.0,5.0,SE_SCREENING_0000001,1.510880e+09,1.510880e+09,27.0,2 - 4 times / month,3-4 drinks,3-11 days in the past year,1 time / week,1 time / week,1 time / week,No,NaN,social_history_888MS005_v009.csv
1,100025931,275.0,5.0,SE_VISIT1_0000002,1.517792e+09,NaN,28.0,2 - 4 times / month,3-4 drinks,3-11 days in the past year,1 time / week,1 time / week,1 time / week,No,NaN,social_history_888MS005_v009.csv
2,100025931,275.0,5.0,SE_VISIT2_0000003,1.549328e+09,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,social_history_888MS005_v009.csv
3,100026381,275.0,5.0,SE_SCREENING_0000001,1.511399e+09,1.511399e+09,22.0,2 - 4 times / month,3-4 drinks,1 or 2 days in the past year,5 or more times / week,5 or more times / week,5 or more times / week,No,NaN,social_history_888MS005_v009.csv
4,100026381,275.0,5.0,SE_VISIT1_0000002,1.515287e+09,NaN,22.0,2 - 4 times / month,3-4 drinks,3-11 days in the past year,5 or more times / week,5 or more times / week,5 or more times / week,No,NaN,social_history_888MS005_v009.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5093,500003862,396.0,5.0,SE_VISIT1_0000002,1.581642e+09,NaN,26.0,2 - 3 times / week,1 drink,3-11 days in the past year,NaN,1 time / week,3 times / week,No,NaN,social_history_888MS005_v012.csv
5094,500003873,396.0,5.0,SE_SCREENING_0000001,1.577495e+09,NaN,23.0,2 - 4 times / month,2 drinks,1 or 2 days in the past year,2 times / week,1 time / week,5 or more times / week,No,NaN,social_history_888MS005_v012.csv
5095,500003873,396.0,5.0,SE_VISIT1_0000002,1.581642e+09,NaN,23.0,2 - 4 times / month,2 drinks,1 or 2 days in the past year,4 times / week,5 or more times / week,5 or more times / week,No,NaN,social_history_888MS005_v012.csv
5096,500003890,396.0,5.0,SE_SCREENING_0000001,1.588208e+09,NaN,24.0,Monthly or less,2 drinks,3-11 days in the past year,2 times / week,5 or more times / week,5 or more times / week,No,NaN,social_history_888MS005_v012.csv


EMR Medications

,mpi


MRI Quantitative Metrics

,mpi,site,study_id,sty_date,pri_date,run_by,prog_ver,rad_acpt,bpf,bpf_chg,...,segcomm,scanner,dgmf,cgmf,thalf,t2overbv,t1dcpp,flrdcpp,nt2lesgt,file
0,100025931,275.0,5.0,1517788800.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.870854,NaN,...,NaN,Prisma_fit,0.028717,0.377815,0.010072,0.000011,N,N,NaN,mri_quantitative_metrics_888MS005_v015.csv
1,100026381,275.0,5.0,1515283200.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.881223,NaN,...,NaN,Prisma_fit,0.028676,0.381350,0.009173,0.000546,N,N,NaN,mri_quantitative_metrics_888MS005_v015.csv
2,100027750,275.0,5.0,1515801600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.837570,NaN,...,NaN,Prisma_fit,0.026422,0.362570,0.009514,0.000462,N,N,NaN,mri_quantitative_metrics_888MS005_v015.csv
3,100027750,275.0,5.0,1547251200.0,1515801600.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.842286,0.56301,...,NaN,Prisma_fit,0.026388,0.361113,0.009668,0.000458,N,N,0.0,mri_quantitative_metrics_888MS005_v015.csv
4,100027750,275.0,5.0,1578700800.0,1547251200.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.845576,0.39063,...,NaN,Prisma_fit,0.026636,0.367079,0.009552,0.000698,N,N,0.0,mri_quantitative_metrics_888MS005_v015.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,500012488,396.0,NaN,1660867200,1628467200.0,BIIB,MSPiev2.0.0,NaN,0.862737,0.06640,...,NaN,Prisma,0.029615,0.343007,0.010948,0.000535,N,N,0.0,mri_quantitative_metrics_888MS005_v017.csv
1256,500012765,326.0,NaN,1603238400,NaN,BIIB,MSPiev2.0.0,NaN,0.868805,NaN,...,NaN,TrioTim,0.030529,0.377447,0.010591,0.000360,N,N,NaN,mri_quantitative_metrics_888MS005_v017.csv
1257,500012785,326.0,NaN,1603843200,NaN,BIIB,MSPiev2.0.0,NaN,0.851414,NaN,...,NaN,TrioTim,0.026835,0.340900,0.008881,0.000494,N,N,NaN,mri_quantitative_metrics_888MS005_v017.csv
1258,500012786,326.0,NaN,1603843200,NaN,BIIB,MSPiev2.0.0,NaN,0.866766,NaN,...,NaN,TrioTim,0.027516,0.345073,0.009826,0.002985,N,N,NaN,mri_quantitative_metrics_888MS005_v017.csv


Combine patients and HCs

In [8]:
mri_df = pd.concat((prepared_tables["MRI Quantitative Metrics"], prepared_tables_hc["MRI Quantitative Metrics"]))

# Convert Date-Column to same format everywhere
# 1. Identify which entries are numeric (Unix timestamps)
is_timestamp = pd.to_numeric(mri_df['sty_date'], errors='coerce').notna()

# 2. Convert timestamps (assumed to be seconds) to datetime
mri_df.loc[is_timestamp, 'date'] = pd.to_datetime(
    mri_df.loc[is_timestamp, 'sty_date'].astype(float), unit='s'
)

# 3. Convert the remaining string dates to datetime
mri_df.loc[~is_timestamp, 'date'] = pd.to_datetime(mri_df.loc[~is_timestamp, 'sty_date'])

# 4. Ensure the column dtype is datetime64[ns]
mri_df['date'] = pd.to_datetime(mri_df['date'])

mri_df = mri_df.merge(sessions_df, left_on=['mpi', 'date'], right_on=['subject', 'acq_time'], how='right').dropna(subset=['session_id', 'subject']).drop_duplicates(subset=['date', 'mpi'])
mri_df.query('session_id == "ses-001"').to_csv('baseline_quantiative_metrics.csv') 

mri_df

,mpi,site,sty_date,pri_date,run_by,prog_ver,rad_acpt,bpf,bpf_chg,t2lesvol,...,t2overbv,t1dcpp,flrdcpp,nt2lesgt,file,study_id,date,session_id,acq_time,subject
0,100052186,225.0,1556841600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.844728,NaN,38.2606,...,0.029835,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-05-03,ses-001,2019-05-03,100052186
6,100052186,225.0,1582329600.0,1556841600.0,BIIB,MSPiev2.0.0,NaN,0.837414,-0.86593,33.4038,...,0.026759,N,N,8.0,mri_quantitative_metrics_v017.csv,NaN,2020-02-22,ses-002,2020-02-22,100052186
8,100006427,285.0,1486598400.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.797080,NaN,34.3370,...,0.040321,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2017-02-09,ses-001,2017-02-09,100006427
19,100006427,285.0,1496880000.0,1486598400.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.801253,0.52353,33.2679,...,0.039202,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-06-08,ses-002,2017-06-08,100006427
28,500002712,326.0,1604793600.0,NaN,BIIB,MSPiev2.0.0,NaN,0.788259,NaN,29.6135,...,0.028293,N,N,NaN,mri_quantitative_metrics_v015.csv,NaN,2020-11-08,ses-001,2020-11-08,500002712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193759,100003594,275.0,1511481600.0,1477612800.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.856995,-0.18956,10.8606,...,0.009806,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-11-24,ses-002,2017-11-24,100003594
193769,100003594,275.0,1577577600.0,1511481600.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.854145,-0.33259,10.4541,...,0.009512,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2019-12-29,ses-003,2019-12-29,100003594
193776,500003493,396.0,1560729600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.873024,NaN,0.1865,...,0.000172,Y,Y,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-06-17,ses-001,2019-06-17,500003493
193782,500003493,396.0,1594339200.0,1560729600.0,BIIB,MSPiev2.0.0,NaN,0.869819,-0.36713,0.2813,...,0.000257,N,N,0.0,mri_quantitative_metrics_v015.csv,NaN,2020-07-10,ses-002,2020-07-10,500003493


Now match MRI-QUantiative values with our sessions. 

 => maybe we should reverse this. 
Currently we get 26782 sessions but have 30750 in the BIDS-Dataset => What happened here?
Should identify


## MSPT patient Characteristics

In This table we find essential things like MS-Type, PDDS-score, etc.



In [19]:
patchar_df = prepared_tables["MSPT Patientcharacteristics"]
patchar_df["date"] = pd.to_datetime(patchar_df.encounter_date, unit='s')
patchar_df.drop_duplicates(inplace=True)



def find_best_characteristics(mpi, date, progress, task):
    progress.update(task, advance=1)
    
    patient_df = patchar_df.query(f'mpi == "{mpi}"')

    patient_df["distance"] = abs(patient_df.date - date)
    patient_df.sort_values('distance', inplace=True)

    if len(patient_df) == 0:
        return  pd.Series([
        
            pd.NA, pd.NA, pd.NA
        ])

    
    best_result = patient_df.iloc[0,:]
    return pd.Series([
        best_result["date"],
        best_result["mstype"],
        best_result["pdds_scr"]
         ])



with Progress() as progress:
    task = progress.add_task("Matching Sessions", total=len(mri_df))
    mri_df[["characteristics_date", "mstype", "pdds_scr"]] = mri_df.apply(lambda x: find_best_characteristics(x.mpi, x.date, progress, task), axis=1 )
   

Output()

In [22]:
mri_df.to_hdf('mri_data.hdf5', key='mri_df')

/tmp/ipykernel_456658/3448646742.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['mpi', 'sty_date', 'pri_date', 'run_by', 'prog_ver', 'segvisqc',
       'segcomm', 'scanner', 't1dcpp', 'flrdcpp', 'file', 'session_id',
       'subject', 'characteristics_date', 'mstype', 'pdds_scr'],
      dtype='object')]

  mri_df.to_hdf('mri_data.hdf5', key='mri_df')


In [23]:
mri_df

,mpi,site,sty_date,pri_date,run_by,prog_ver,rad_acpt,bpf,bpf_chg,t2lesvol,...,nt2lesgt,file,study_id,date,session_id,acq_time,subject,characteristics_date,mstype,pdds_scr
0,100052186,225.0,1556841600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.844728,NaN,38.2606,...,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-05-03,ses-001,2019-05-03,100052186,<NA>,<NA>,<NA>
6,100052186,225.0,1582329600.0,1556841600.0,BIIB,MSPiev2.0.0,NaN,0.837414,-0.86593,33.4038,...,8.0,mri_quantitative_metrics_v017.csv,NaN,2020-02-22,ses-002,2020-02-22,100052186,<NA>,<NA>,<NA>
8,100006427,285.0,1486598400.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.797080,NaN,34.3370,...,NaN,mri_quantitative_metrics_v015.csv,NaN,2017-02-09,ses-001,2017-02-09,100006427,2017-02-09 14:15:19,Relapsing Remitting MS,0.0
19,100006427,285.0,1496880000.0,1486598400.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.801253,0.52353,33.2679,...,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-06-08,ses-002,2017-06-08,100006427,2017-02-09 14:15:19,Relapsing Remitting MS,0.0
28,500002712,326.0,1604793600.0,NaN,BIIB,MSPiev2.0.0,NaN,0.788259,NaN,29.6135,...,NaN,mri_quantitative_metrics_v015.csv,NaN,2020-11-08,ses-001,2020-11-08,500002712,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193759,100003594,275.0,1511481600.0,1477612800.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.856995,-0.18956,10.8606,...,0.0,mri_quantitative_metrics_v015.csv,NaN,2017-11-24,ses-002,2017-11-24,100003594,2017-06-02 14:48:29,Relapsing Remitting MS,0.0
193769,100003594,275.0,1577577600.0,1511481600.0,BIIB,MSPiev2.0.0rc.p1,NaN,0.854145,-0.33259,10.4541,...,0.0,mri_quantitative_metrics_v015.csv,NaN,2019-12-29,ses-003,2019-12-29,100003594,2019-12-29 14:34:51,NaN,0.0
193776,500003493,396.0,1560729600.0,NaN,BIIB,MSPiev2.0.0rc.p1,NaN,0.873024,NaN,0.1865,...,NaN,mri_quantitative_metrics_v015.csv,NaN,2019-06-17,ses-001,2019-06-17,500003493,2019-06-20 14:53:41,Relapsing Remitting MS,0.0
193782,500003493,396.0,1594339200.0,1560729600.0,BIIB,MSPiev2.0.0,NaN,0.869819,-0.36713,0.2813,...,0.0,mri_quantitative_metrics_v015.csv,NaN,2020-07-10,ses-002,2020-07-10,500003493,2020-07-19 11:49:10,Clinically Isolated Syndrome,0.0
